# Fun with light

Suppose we have a robot that's goal is to seek light.  In the world there is a single light source which is attached to another robot. The second robot's goal is to keep moving and avoid obstacles.  What happens when we put these two robots together?  Let's find out...

In [15]:
from aitk.robots import World, Scribbler, RangeSensor, LightSensor, Bulb, __version__
__version__

'0.9.2'

In [16]:
world = World(width=150, height=150, scale=3.0)

Random seed set to: 3962240


In [17]:
avoider = Scribbler(x=50, y=40, a=135, color="red")
ir1 = RangeSensor(position=(6,-6),width=57.3,max=20,angle=90,name="left-ir")
ir2 = RangeSensor(position=(6,6),width=57.3,max=20,angle=90,name="right-ir")
avoider.add_device(ir1)
avoider.add_device(ir2)
avoider.add_device(Bulb("yellow",brightness=50))
world.add_robot(avoider)

In [18]:
seeker = Scribbler(x=60, y=100, a=270, color="yellow")
seeker.add_device(LightSensor(position=(6,-6), name="left-light"))
seeker.add_device(LightSensor(position=(6,6), name="right-light"))
world.add_robot(seeker)

In [19]:
world.watch()

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…

In [20]:
from random import random

In [21]:
def wander_avoid(robot):
    max_val = robot["left-ir"].get_max()
    if robot["left-ir"].get_distance() < max_val:
        robot.move(0.1, -0.3)
    elif robot["right-ir"].get_distance() < max_val:
        robot.move(0.1, 0.3)
    else:
        robot.move(1, random()-0.5)

In [24]:
def seek_light(robot):
    robot.speak("")
    left_light = robot["left-light"].get_brightness()
    right_light = robot["right-light"].get_brightness()
    diff = left_light - right_light
    if robot.stalled:
        robot.reverse()
    elif left_light + right_light > 1.9:
        robot.move(0,0)
        robot.speak("gotcha")
    elif diff > 0.01:
        robot.move(1,0.3)
    elif diff < -0.01:
        robot.move(1,-0.3)   
    elif robot.world.time%2 == 0.0:
        robot.move(1.0, random()*0.5-0.25)

In [25]:
world.reset()
world.set_seed(random())
world.run([wander_avoid,seek_light],real_time=True)

Using random seed: 0.37299892226419673
Using random seed: 0.04949634142924597
Simulation stopped at: 00:00:45.2; speed 0.97 x real time
